In [414]:
from random import randint, shuffle, random
import numpy as np

# Matrice d'adjacence

In [415]:
# Génération de la matrice adjacence à partir d'un fichier
def load_matrix():
    mat = []
    with open('mat_adjacence', 'r') as file:   
        for l in file.readlines():  
            val = [int(x) for x in l.split()]
            mat.append(val)            
    return mat

# Créer une matrice d'adjacence dont le cout du circuit Hamiltonien optimal est celui de la périphérie  
def create_matrix(n_ville):
    mat = np.zeros((n_ville, n_ville))
    
    for i in range(n_ville):
        for j in range(i, n_ville):
            cout = randint(1, 10)
            if i == j:
                cout = 0
            i_after = i+1 if i+1 < n_ville else 0
            if i_after == j:
                cout = 1
                 
            mat[i][j] = cout
            mat[j][i] = cout
    mat[n_ville-1][0] = 1
    mat[0][n_ville-1] = 1

    return mat

Initialisation de la population

In [416]:
def population_init(nb_ville, size):
    population = []
    
    for _ in range(size):
        circuit = list(range(nb_ville))
        shuffle(circuit) # circuit random
        population.append([circuit, 5])
        
    return population



In [417]:

def empty_mat(width):
    return [[0]*width for _ in range(width)]

class Agent:
    def __init__(self, nodes_to_explore, deposit_Q, prob_explore_Y=1, alpha=1, beta=1):
        self.starting_node = 0 # Toute les fourmies commencent au même endroit
        self.deposit_Q = deposit_Q
        self.null_prob = prob_explore_Y 
        # Pour la decision de l'exploration
        self.alpha = 1 # Importance des phéromone
        self.beta = 1 # Importance de la distance d'une ville
        self.reset_exploration(list(nodes_to_explore)) 
    
     
    def reset_exploration(self, nodes_to_explore):
        self.unexplored = nodes_to_explore
        self.unexplored.remove(self.starting_node)
        self.explored = [self.starting_node]
    
    def get_travel_length(self, mat_cout):
        travel_length = 0
        for i in range(1, len(self.explored)):
            depart = self.explored[i-1]
            arrival = self.explored[i]
            
            travel_length += mat_cout[depart][arrival]
            
        return travel_length
    
    def deposite_pheromone(self, mat_cost, pheromone_mat):
        
        pheromone_quantity = self.deposit_Q /  self.get_travel_length(mat_cost)
        
        
        for i in range(1, len(self.explored)):
            depart = self.explored[i-1]
            arrival = self.explored[i]
            
            # On dépose les phéromones sur les arrettes
            pheromone_mat[depart][arrival] += pheromone_quantity
          
          
      
    def choose_next_location(self, mat_cost, mat_pheromone):
        # On choisit la prochaine ville a explorer aléatoirement selon les phéromones,
        # avec une proba non nul qu'on explore une ville jamais exploré par les agents
        # Et retourne un indice parmis la liste des unexplored
        
        depart = self.explored[-1] 
        sum = 0
        
        # On somme les differente pheromone pour avoir les probabilité
        for arrival in self.unexplored:
            
            # On a une proba non nul pour qu'un agent puisse explorer une ville jamais exploré par les autres agents controllé par le parametre Y
            sum += self.null_prob \
                + mat_pheromone[depart][arrival] ** self.alpha \
                + 1/mat_cost[depart][arrival] ** self.beta # ηij la visibilité, qui est égale à l’inverse de la distance de deux villes i et j 
        
        # random qui decide de la prochaine ville à explorer
        r = random() *  sum
        s, i = 0, 0  
        # Recherche de la ville qui vient d'etre choisit
        for arrival in self.unexplored: 
            s += self.null_prob \
                + mat_pheromone[depart][arrival] ** self.alpha \
                + 1/mat_cost[depart][arrival] ** self.beta
            
            # On s'arrette lorsque l'on a trouvé la prochaine ville, et on retourne son indice dans unexplored
            if s >= r:
                return i
            
            i += 1
        return i

    
    """def advance(self, mat_cost, mat_pheromone): # avance l'agent
        
        # Si on a atteint toutes les villes 
        if len(self.unexplored) <= 0:
            # Une fois la tournée terminé, l'agent dépose les phéromones
            self.deposite_pheromone(mat_cost, mat_pheromone)
            # On reset les villes explorer et on recommence l'exploration 
            self.reset_exploration(self.explored)
         
        # On choisit la prochaine destination parmis ceux qui reste à explorer 
        next_i = self.choose_next_location(mat_cost, mat_pheromone) 
        curr_node = self.unexplored.pop(next_i)
        self.explored.append(curr_node)"""
        
    def explore(self, mat_cost, mat_pheromone):
        # Tant qu'on a pas tout exploré
        while len(self.unexplored) > 0:
            # On choisit la prochaine destination parmis ceux qui reste à explorer 
            next_i = self.choose_next_location(mat_cost, mat_pheromone) 
            curr_node = self.unexplored.pop(next_i)
            self.explored.append(curr_node)
            
        # Une fois la tournée terminé, l'agent dépose les phéromones
        self.deposite_pheromone(mat_cost, mat_pheromone)
        # On reset les villes explorer pour pouvoir recommencer l'exploration 
        self.reset_exploration(self.explored)
        
def evaporation(pheromone_mat, disipation_rate):
    for i in range(len(pheromone_mat)):
        for j in range(len(pheromone_mat[0])):
            if pheromone_mat[i][j] > 0:
                pheromone_mat[i][j] -= disipation_rate
                
            

In [418]:
def index_max(arr, exclude_indices): # retourne l'indice dont la valeur est maximal, et qui n'est pas dans exclude_indices
    
    # On n'explore pas a partir du premier, au cas où l'élément 0 fait partir de exclude_indices
    best_i = None 
    for i in range(len(arr)): 
        if best_i == None:
            if i not in exclude_indices: 
                best_i = i
            continue 
        
        if arr[best_i] < arr[i] and i not in exclude_indices:
            best_i = i
            
    return best_i

def find_best(pheromone_mat): # Trouve le chemin le plus parcourus
    res = [0]
    i = 1
    
    while i < len(pheromone_mat):
        
        depart = res[-1]
        arrival = index_max(pheromone_mat[depart], res)
        res.append(arrival)
        
        i += 1
        
    return res

def cost(cost_mat, path): # Renvoie le cout d'un chemin, par rapport à la matrice d'adjacence
    total_cost = 0
    
    for i in range(1, len(path)):
        depart = path[i - 1]
        arrival = path[i]
        total_cost += cost_mat[depart][arrival]
    
    return total_cost  

def main():
    size = 10
    epoch_max = 100
    agents_number = 10 
    disipation_rate = 1
    deposit_Q = 1
    prob_explore_Y = 1
    
    mat_cost = create_matrix(size)
    print("matrice d'adjacence")
    print(mat_cost)
    pheromone_mat = empty_mat(size)
    agents = [Agent(list(range(size)), deposit_Q, prob_explore_Y) for _ in range(agents_number)]
    
    disipation_rate = 0
    
    for i in range(epoch_max): 
        
        #print("iteration: ", i)
        # On avance chaque agents dans leurs exploration
        for agent in agents:
            agent.explore(mat_cost, pheromone_mat) # Un agent avance d'un noeud
        # On évapore les phéromones a chaque fin d'itération
        evaporation(pheromone_mat, disipation_rate)
    
    best_path = find_best(pheromone_mat) 
    print("Meilleur chemin: ", best_path)
    print("Cout: ", cost(mat_cost, best_path))
    
main()

[[ 0.  1.  4.  6.  9. 10.  5.  9.  5.  1.]
 [ 1.  0.  1.  9.  3. 10. 10. 10.  3.  3.]
 [ 4.  1.  0.  1.  2.  7.  7.  8.  8.  9.]
 [ 6.  9.  1.  0.  1.  8.  5.  7.  8.  5.]
 [ 9.  3.  2.  1.  0.  1.  9. 10.  4.  6.]
 [10. 10.  7.  8.  1.  0.  1.  7.  7.  3.]
 [ 5. 10.  7.  5.  9.  1.  0.  1.  8.  6.]
 [ 9. 10.  8.  7. 10.  7.  1.  0.  1.  9.]
 [ 5.  3.  8.  8.  4.  7.  8.  1.  0.  1.]
 [ 1.  3.  9.  5.  6.  3.  6.  9.  1.  0.]]
Meilleur chemin:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Cout:  9.0
